In [9]:
import random

def generate_data(num_entries, num_rows, filename):
    # Open the file for writing
    with open(filename, 'w') as file:
        # Generate each row
        for _ in range(num_rows):
            # Generate each entry in the row as a list of 3 random integers
            row = [f"[{random.randint(10, 60)}, {random.randint(10, 60)}, {random.randint(10, 60)}]" for _ in range(num_entries)]
            # Join all entries into a single string with commas and write to file
            formatted_row = ', '.join(row)
            file.write(f"\"{formatted_row}\"\n")

# Example usage:
generate_data(4, 6, 'fake_data.csv')

In [2]:
import numpy as np
import pandas as pd
import ast

# Load and parse the CSV data
def load_and_parse_data(file_path):
    data = pd.read_csv(file_path, header=None)
    sequences = []
    for index, row in data.iterrows():
        # Convert the row from string representation to list of lists of integers
        sequence = ast.literal_eval(row[0])
        sequences.append(sequence)
    return np.array(sequences)

# Split sequences into input and target
def split_sequences(sequences):
    inputs = []
    targets = []
    for seq in sequences:
        mid = len(seq) // 2
        inputs.append(seq[:mid])
        targets.append(seq[mid:])
    return np.array(inputs), np.array(targets)

# Load and preprocess the data
file_path = 'fake_data.csv'  # replace with your file path
sequences = load_and_parse_data(file_path)
inputs, targets = split_sequences(sequences)


In [3]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define the model parameters
num_features = 3  # Since each element in the sequence is a list of 3 integers
latent_dim = 50  # Latent dimensionality of the encoding space

# Define the encoder
encoder_inputs = Input(shape=(None, num_features))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None, num_features))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_features, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 3)]            0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 3)]            0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 50),                 10800     ['input_1[0][0]']             
                              (None, 50),                                                         
                              (None, 50)]                                                         
                                                                                              

In [4]:
# Prepare training data
encoder_input_data = inputs
decoder_input_data = np.zeros_like(targets)
decoder_input_data[:, 1:, :] = targets[:, :-1, :]  # Shifted target sequences
decoder_target_data = targets

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=50, validation_split=0.2)


Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 1548.6641 - val_loss: 1542.0903
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 1542.0903 - val_loss: 1535.9143
Epoch 3/50
1/1 [==============================] - 0s 36ms/step - loss: 1535.9144 - val_loss: 1530.0850
Epoch 4/50
1/1 [==============================] - 0s 36ms/step - loss: 1530.0851 - val_loss: 1524.5430
Epoch 5/50
1/1 [==============================] - 0s 33ms/step - loss: 1524.5430 - val_loss: 1519.2385
Epoch 6/50
1/1 [==============================] - 0s 35ms/step - loss: 1519.2384 - val_loss: 1514.1326
Epoch 7/50
1/1 [==============================] - 0s 37ms/step - loss: 1514.1324 - val_loss: 1509.1918
Epoch 8/50
1/1 [==============================] - 0s 30ms/step - loss: 1509.1918 - val_loss: 1504.3862
Epoch 9/50
1/1 [==============================] - 0s 33ms/step - loss: 1504.3862 - val_loss: 1499.6866
Epoch 10/50
1/1 [==============================] - 0s 36ms/step - loss: 149

In [5]:
# Define the encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Function to generate sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence
    target_seq = np.zeros((1, 1, num_features))

    # Populate the first element with a start token
    target_seq[0, 0, :] = [0, 0, 0]

    # Sampling loop
    stop_condition = False
    decoded_sequence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Append the predicted token to the decoded sequence
        decoded_sequence.append(output_tokens[0, 0, :])

        # Exit condition
        if len(decoded_sequence) >= input_seq.shape[1]:
            stop_condition = True

        # Update the target sequence
        target_seq = np.zeros((1, 1, num_features))
        target_seq[0, 0, :] = output_tokens[0, 0, :]

        # Update states
        states_value = [h, c]

    return np.array(decoded_sequence)

# Example usage
test_input = encoder_input_data[0:1]  # Replace with actual test input
decoded_sequence = decode_sequence(test_input)
print("Decoded sequence:", decoded_sequence)


1/1 [==============================] - 0s 24ms/step
Decoded sequence: [[3.389219  3.5514295 4.4847946]
 [3.8782923 4.0184774 4.7389703]
 [3.9243848 4.3190184 4.7797847]
 [3.861138  4.4353952 4.760472 ]]
